In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data =  pd.read_csv('../input/train.csv',nrows = 10_000_000)
train_data.head()

In [ ]:
train_data.info()

In [ ]:
print('check for null values:- \n', (train_data.isnull().sum()))

In [ ]:
train_data = train_data.dropna(axis = 0)
print('after deleting null valuse data: ', train_data.isnull().sum())


In [ ]:
def clean_df(data):
    return data[(data.fare_amount > 0) & 
            (data.passenger_count > 0) & (data.passenger_count < 10)]
train_data = clean_df(train_data)


In [ ]:
def distance(lat1, lon1, lat2, lon2):
     earth_radius=6371
     lat1, lon1, lat2, lon2 = map(np.radians,[lat1, lon1, lat2, lon2])
     a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
     return earth_radius * 2 * np.arcsin(np.sqrt(a))

train_data['total distance'] = distance(train_data['pickup_latitude'],train_data['pickup_longitude'],train_data['dropoff_latitude'] , train_data['dropoff_longitude'])
train_data.head(50)

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(train_data.corr(),annot=True)

In [ ]:
sns.pairplot(train_data)

In [ ]:
X = train_data[['passenger_count','total distance']]
y = train_data['fare_amount']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X= scaler.transform(X)

In [ ]:
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,  X_test,y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression 
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_predict= regressor.predict(X_test)

In [ ]:
# I don't know why I am getting horizontal line of actual fare_rate:- 

test = pd.DataFrame({'Predicted':y_predict,'Actual':y_test})
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.figure(figsize=(18,12))
plt.plot(test[:200])
plt.legend(['Actual','Predicted'])
sns.jointplot(x='Actual',y='Predicted',data=test,kind='reg')

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

score_test = r2_score(regressor.predict(X_test),y_test)
print(score_test)
                                                   